In [ ]:
import re
import string
import tensorflow as tf
import numpy as np
import pandas as pd

from model import Word2Vec

from tensorflow.keras.layers import Embedding, Dot
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_csv("data/pokemon_cleansed.csv")
df

,english_name,description
0,bulbasaur,grass seed pokémon there is a plant seed on b...
1,ivysaur,grass seed pokémon when the bulb on ivysaurs ...
2,venusaur,grass seed pokémon venusaurs plant blooms whe...
3,charmander,fire lizard pokémon charmander has a preferen...
4,charmeleon,fire flame pokémon charmeleon has a barbaric ...
...,...,...
146,dratini,dragon dragon pokémon dratini dwells near bodi...
147,dragonair,dragon dragon pokémon dragonair lives in prist...
148,dragonite,dragon dragon pokémon dragonite is a kindhear...
149,mewtwo,psychic genetic pokémon mewtwos dna is almost...


In [22]:
# Put together a training corpus
descriptions = list(df["description"].values)

# Clean and tokenize text
def preprocess_text(text):
    text = text.lower()  # lowercase
    text = re.sub(f"[{string.punctuation}]", "", text)  # remove punctuation
    words = text.split()  # tokenize into words
    return words

# Apply to all sentences
dataset = [preprocess_text(sentence) for sentence in descriptions]
print(dataset)

[['grass', 'seed', 'pokémon', 'there', 'is', 'a', 'plant', 'seed', 'on', 'bulbasaurs', 'back', 'right', 'from', 'the', 'day', 'bulbasaur', 'is', 'born', 'the', 'seed', 'slowly', 'grows', 'larger'], ['grass', 'seed', 'pokémon', 'when', 'the', 'bulb', 'on', 'ivysaurs', 'back', 'grows', 'large', 'ivysaur', 'appears', 'to', 'lose', 'the', 'abil', 'ivysaury', 'to', 'stand', 'on', 'ivysaurs', 'hind', 'legs'], ['grass', 'seed', 'pokémon', 'venusaurs', 'plant', 'blooms', 'when', 'venusaur', 'is', 'absorbing', 'solar', 'energy', 'venusaur', 'stays', 'on', 'the', 'move', 'to', 'seek', 'sunlight'], ['fire', 'lizard', 'pokémon', 'charmander', 'has', 'a', 'preference', 'for', 'hot', 'things', 'when', 'charmander', 'rains', 'steam', 'is', 'said', 'to', 'spout', 'from', 'the', 'tip', 'of', 'charmanders', 'tail'], ['fire', 'flame', 'pokémon', 'charmeleon', 'has', 'a', 'barbaric', 'nature', 'in', 'battle', 'charmeleon', 'whips', 'charmeleons', 'fiery', 'tail', 'around', 'and', 'slashes', 'away', 'w', '

In [23]:
window_size = 10
embedding_dim = 20

epochs = 50
optimizer = Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

word2vec = Word2Vec(dataset, embedding_dim)
train_dataset = word2vec.prepare_dataset(window_size)
word2vec.compile(optimizer, loss)
word2vec.fit(train_dataset, epochs=epochs)

Word2Vec vocabulary size: 1471
Word2Vec vocabulary words and indexes: {'quick': 0, 'ideal': 1, 'almost': 2, 'zapdos': 3, 'studied': 4, 'turns': 5, 'born': 6, 'roll': 7, 'see': 8, 'flying': 9, 'belly': 10, 'and': 11, 'machop': 12, 'flareon': 13, 'insect': 14, 'terms': 15, 'horseas': 16, 'together': 17, 'classy': 18, 'slightest': 19, 'seeds': 20, 'quickly': 21, 'climate': 22, 'sharp': 23, 'dugtrio': 24, 'out': 25, 'jolteons': 26, 'first': 27, 'golbats': 28, 'clefable': 29, 'six': 30, 'vastly': 31, 'escape': 32, 'communicates': 33, 'toxic': 34, '1700': 35, 'person': 36, 'recognized': 37, 'prideful': 38, 'pincers': 39, 'horseachblack': 40, 'tears': 41, 'raichuself': 42, 'rhydon': 43, 'nidorino': 44, 'showing': 45, 'squirts': 46, 'tender': 47, 'belt': 48, 'goldeens': 49, 'sense': 50, 'molt': 51, 'lizard': 52, 'golducks': 53, 'fox': 54, 'bathing': 55, 'lung': 56, 'slowpokes': 57, 'waves': 58, 'pidgeottos': 59, 'keeper': 60, 'jellyfish': 61, 'takes': 62, 'taurosself': 63, 'fly': 64, 'comes': 

In [24]:
word2vec.summary()

Model: "word2_vec_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 word_embedding (Embedding)  multiple                  29420     
                                                                 
 dot_8 (Dot)                 multiple                  0         
                                                                 
Total params: 29420 (114.92 KB)
Trainable params: 29420 (114.92 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
# Example usage
word2vec.get_word_embedding("pikachu")

array([-0.8940277 ,  0.9868017 , -1.0189716 ,  0.9820899 , -1.1174933 ,
       -1.0046421 , -0.60882014,  0.8898803 , -1.0056114 , -0.44581392,
        0.69638526, -0.98456955,  1.0080854 ,  0.7706229 ,  0.6281304 ,
       -0.2901547 , -0.83576596, -0.98115563,  0.895597  ,  0.8978242 ],
      dtype=float32)

In [49]:
# Example usage
word2vec.compute_similarity("bulbasaur", "seed")

16.817562

In [50]:
# Example usage
word2vec.compute_similarity("bulbasaur", "fire")

18.500116